Importing the library

In [1]:
import io
import sys
import os
import numpy as np
import pandas as pd
import nltk
import csv, collections
from textblob import TextBlob
from sklearn.utils import shuffle
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
import pickle
import replace_emoji

Define a class to load the SentimentWordnet and write methods to calculate the scores

In [2]:
class load_senti_word_net(object):
    """
    constructor to load the file and read the file as CSV
    6 columns - pos, ID, PosScore, NegScore, synsetTerms, gloss
    synsetTerms can have multiple similar words like abducting#1 abducent#1 and will read each one and calculaye the scores
    """
    
    def __init__(self):
        sent_scores = collections.defaultdict(list)
        with io.open("C:\\Users\\saksham raj seth\\Desktop\\SentiWordNet_3.0.0_20130122.txt") as fname:
            file_content = csv.reader(fname, delimiter='\t',quotechar='"')
            for line in file_content:                
                if line[0].startswith('#') :
                    continue                    
                pos, ID, PosScore, NegScore, synsetTerms, gloss = line 
                for terms in synsetTerms.split(" "):
                    term = terms.split("#")[0]
                    term = term.replace("-","").replace("_","")
                    key = "%s/%s"%(pos,term.split("#")[0])
                    try:
                        sent_scores[key].append((float(PosScore),float(NegScore)))
                    except:
                        sent_scores[key].append((0,0))
                    
        for key, value in sent_scores.items():
            sent_scores[key] = np.mean(value,axis=0)
        
        self.sent_scores = sent_scores    
     
    
    def score_word(self, word):
        pos = nltk.pos_tag([word])[0][1]
        return self.score(word, pos)
    
    def score(self,word, pos):
        """
        Identify the type of POS, get the score from the senti_scores and return the score
        """
        
        if pos[0:2] == 'NN':
            pos_type = 'n'
        elif pos[0:2] == 'JJ':
            pos_type = 'a'
        elif pos[0:2] =='VB':
            pos_type='v'
        elif pos[0:2] =='RB':
            pos_type = 'r'
        else:
            pos_type =  0
            
        if pos_type != 0 :    
            loc = pos_type+'/'+word
            score = self.sent_scores[loc]
            if len(score)>1:
                return score
            else:
                return np.array([0.0,0.0])
        else:
            return np.array([0.0,0.0])
        
    """
    Repeat the same for a sentence
    nltk.pos_tag(word_tokenize("My name is Suraj"))
    [('My', 'PRP$'), ('name', 'NN'), ('is', 'VBZ'), ('Suraj', 'NNP')]    
    """    
        
    def score_sentencce(self, sentence):
        pos = nltk.pos_tag(sentence)
        print (pos)
        mean_score = np.array([0.0, 0.0])
        for i in range(len(pos)):
            mean_score += self.score(pos[i][0], pos[i][1])
            
        return mean_score
    
    def pos_vector(self, sentence):
        pos_tag = nltk.pos_tag(sentence)
        vector = np.zeros(4)
        
        for i in range(0, len(pos_tag)):
            pos = pos_tag[i][1]
            if pos[0:2]=='NN':
                vector[0] += 1
            elif pos[0:2] =='JJ':
                vector[1] += 1
            elif pos[0:2] =='VB':
                vector[2] += 1
            elif pos[0:2] == 'RB':
                vector[3] += 1
                
        return vector
            

Now let's extract the features

###Stemming and Lemmatization

In [3]:
porter = nltk.PorterStemmer()
sentiments = load_senti_word_net()

In [4]:
def gram_features(features,sentence):
    sentence_rep = replace_emoji.replace_reg(str(sentence))
    token = nltk.word_tokenize(sentence_rep)
    token = [porter.stem(i.lower()) for i in token]        
    
    bigrams = nltk.bigrams(token)
    bigrams = [tup[0] + ' ' + tup[1] for tup in bigrams]
    grams = token + bigrams
    #print (grams)
    for t in grams:
        features['contains(%s)'%t]=1.0
    

In [5]:
import string
def sentiment_extract(features, sentence):
    sentence_rep = replace_emoji.replace_reg(sentence)
    token = nltk.tokenize.word_tokenize(sentence_rep)    
    token = [porter.stem(i.lower()) for i in token]   
    mean_sentiment = sentiments.score_sentencce(token)
    features["Positive Sentiment"] = mean_sentiment[0]
    features["Negative Sentiment"] = mean_sentiment[1]
    features["sentiment"] = mean_sentiment[0] - mean_sentiment[1]
    #print(mean_sentiment[0], mean_sentiment[1])
    
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in token]).strip())
        features["Blob Polarity"] = text.sentiment.polarity
        features["Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["Blob Polarity"] = 0
        features["Blob Subjectivity"] = 0
        #print("do nothing")
        
    
    first_half = token[0:int(len(token)/2)]    
    mean_sentiment_half = sentiments.score_sentencce(first_half)
    features["positive Sentiment first half"] = mean_sentiment_half[0]
    features["negative Sentiment first half"] = mean_sentiment_half[1]
    features["first half sentiment"] = mean_sentiment_half[0]-mean_sentiment_half[1]
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in first_half]).strip())
        features["first half Blob Polarity"] = text.sentiment.polarity
        features["first half Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["first Blob Polarity"] = 0
        features["first Blob Subjectivity"] = 0
        print("do nothing")
    
    second_half = token[int(len(token)/2):]
    mean_sentiment_sechalf = sentiments.score_sentencce(second_half)
    features["positive Sentiment second half"] = mean_sentiment_sechalf[0]
    features["negative Sentiment second half"] = mean_sentiment_sechalf[1]
    features["second half sentiment"] = mean_sentiment_sechalf[0]-mean_sentiment_sechalf[1]
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in second_half]).strip())
        features["second half Blob Polarity"] = text.sentiment.polarity
        features["second half Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["second Blob Polarity"] = 0
        features["second Blob Subjectivity"] = 0
        print("do nothing")  
    
    
    

In [6]:
features = {}
sentiment_extract(features,"a long narrow opening")

[('a', 'DT'), ('long', 'JJ'), ('narrow', 'JJ'), ('open', 'JJ')]
[('a', 'DT'), ('long', 'JJ')]
[('narrow', 'JJ'), ('open', 'JJ')]


In [7]:
def pos_features(features,sentence):
    sentence_rep = replace_emoji.replace_reg(sentence)
    token = nltk.word_tokenize(sentence_rep)
    token = [ porter.stem(each.lower()) for each in token]
    pos_vector = sentiments.pos_vector(token)
    for j in range(len(pos_vector)):
        features['POS_'+str(j+1)] = pos_vector[j]
    print ("done")
    

In [8]:
features = {}
pos_features(features,"a long narrow opening")

done


In [9]:
def capitalization(features,sentence):
    count = 0
    for i in range(len(sentence)):
        count += int(sentence[i].isupper())
    features['Capitalization'] = int(count > 3)
    print (count)

In [10]:
features = {}
capitalization(features,"A LoNg NArrow opening")

5


In [11]:
import topic
topic_mod = topic.topic(nbtopic=200,alpha='symmetric')

C:\Users\saksham raj seth\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [12]:
topic_mod = topic.topic(model=os.path.join('C:\\Users\\saksham raj seth\\Documents\\SarcasmDetection-master\\Data\\topics.tp'),dicttp=os.path.join('C:\\Users\\saksham raj seth\\Documents\\SarcasmDetection-master\\Data\\topics_dict.tp'))

In [13]:
def topic_feature(features,sentence,topic_modeler):    
    topics = topic_modeler.transform(sentence)    
    for j in range(len(topics)):
        features['Topic :'] = topics[j][1]
    

In [32]:
#features={}
#topic_feature(features,"A LoNg NArrow opening",topic_mod)

In [14]:
def get_features(sentence, topic_modeler):
    #nltk.download('punkt')
    features = {}
    gram_features(features,sentence)
    pos_features(features,sentence)
    sentiment_extract(features, sentence)
    capitalization(features,sentence)
    #topic_feature(features, sentence,topic_modeler)
    return features

In [169]:
df = pd.DataFrame()
df = pd.read_csv("C:\\Users\\saksham raj seth\\Desktop\\dataset_csv.csv",delimiter = '\t',quoting=3)
df.head()

,"""tweets","label"",,,,,"
0,"""I love working midnights tweet","1"",,,,,"
1,"""I hate when I buy a bag of air and there's ch...","1"",,,,,"
2,"""my grandad always sounds so ill when i speak ...","0"",,,,,"
3,"I realize I'm annoying to everyone, so I won't...","0"",,"
4,"""I love when I find these dudes on vine!! #Fol...","1"",,,,,"


In [16]:
import re
featureset=[]
for i in range(0,df.size):
    temp = str(df["tweets"][i])
    temp = re.sub(r'[^\x00-\x7F]+','',temp)
    featureset.append((get_features(temp,topic_mod), df["label"][i]))

done
[('i', 'NN'), ('love', 'VBP'), ('work', 'NN'), ('midnight', 'NN'), ('tweet', 'NN')]
[('i', 'NNS'), ('love', 'VBP')]
[('work', 'NN'), ('midnight', 'NN'), ('tweet', 'NN')]
1
done
[('i', 'NN'), ('hate', 'VBP'), ('when', 'WRB'), ('i', 'JJ'), ('buy', 'VBP'), ('a', 'DT'), ('bag', 'NN'), ('of', 'IN'), ('air', 'NN'), ('and', 'CC'), ('there', 'EX'), ("'s", 'VBZ'), ('chip', 'NN'), ('in', 'IN'), ('it', 'PRP'), ('#', '#'), ('not', 'RB')]
[('i', 'NN'), ('hate', 'VBP'), ('when', 'WRB'), ('i', 'JJ'), ('buy', 'VBP'), ('a', 'DT'), ('bag', 'NN'), ('of', 'IN')]
[('air', 'NN'), ('and', 'CC'), ('there', 'EX'), ("'s", 'VBZ'), ('chip', 'NN'), ('in', 'IN'), ('it', 'PRP'), ('#', '#'), ('not', 'RB')]
2
done
[('my', 'PRP$'), ('grandad', 'NN'), ('alway', 'RB'), ('sound', 'VBD'), ('so', 'RB'), ('ill', 'JJ'), ('when', 'WRB'), ('i', 'NN'), ('speak', 'VBP'), ('to', 'TO'), ('him', 'PRP'), ('on', 'IN'), ('the', 'DT'), ('phone', 'NN')]
[('my', 'PRP$'), ('grandad', 'NN'), ('alway', 'RB'), ('sound', 'VBD'), ('so', 'R

[('a', 'DT'), ('great', 'JJ'), ('sign', 'NN'), ('#', '#'), ('arsen', 'NN')]
3
done
[('everyth', 'NN'), ('you', 'PRP'), ('need', 'VBP'), ('to', 'TO'), ('know', 'VB'), ('about', 'IN'), ('the', 'DT'), ('new', 'JJ'), ('#', '#'), ('pinterest', 'JJS'), ('analyt', 'NN'), (':', ':'), ('#', '#'), ('socialmedia', 'JJ'), ('#', '#'), ('social', 'JJ'), ('#', '#'), ('media', 'NNS'), ('#', '#'), ('track', 'NN')]
[('everyth', 'NN'), ('you', 'PRP'), ('need', 'VBP'), ('to', 'TO'), ('know', 'VB'), ('about', 'IN'), ('the', 'DT'), ('new', 'JJ'), ('#', '#'), ('pinterest', 'NN')]
[('analyt', 'NN'), (':', ':'), ('#', '#'), ('socialmedia', 'JJ'), ('#', '#'), ('social', 'JJ'), ('#', '#'), ('media', 'NNS'), ('#', '#'), ('track', 'NN')]
10
done
[('when', 'WRB'), ('i', 'NN'), ('speak', 'VBP'), ('politli', 'NN'), ('to', 'TO'), ('peopl', 'VB'), ('they', 'PRP'), ('understand', 'VBP'), ('wrong', 'JJ'), ('so', 'RB'), ('ill', 'JJ'), ('be', 'VB'), ('rude', 'JJ')]
[('when', 'WRB'), ('i', 'NN'), ('speak', 'VBP'), ('politli

[('he', 'PRP'), ("'s", 'VBZ'), ('commit', 'NN'), ('a', 'DT'), ('crime', 'NN'), ('against', 'IN'), ('societi', 'NN'), (',', ','), ('societi', 'JJ'), ('deserv', 'NN'), ('to', 'TO'), ('speak', 'VB'), ('it', 'PRP'), ('mind', 'VB'), ('but', 'CC'), ('should', 'MD'), ('not', 'RB'), ('do', 'VB'), ('so', 'RB'), ('impuls', 'VB'), ('.', '.')]
[('he', 'PRP'), ("'s", 'VBZ'), ('commit', 'NN'), ('a', 'DT'), ('crime', 'NN'), ('against', 'IN'), ('societi', 'NN'), (',', ','), ('societi', 'NN'), ('deserv', 'NN')]
[('to', 'TO'), ('speak', 'VB'), ('it', 'PRP'), ('mind', 'VB'), ('but', 'CC'), ('should', 'MD'), ('not', 'RB'), ('do', 'VB'), ('so', 'RB'), ('impuls', 'VB'), ('.', '.')]
1
done
[('glad', 'VB'), ('the', 'DT'), ('first', 'JJ'), ('floor', 'NN'), ('of', 'IN'), ('potomac', 'NN'), ('smell', 'NN'), ('like', 'IN'), ('straight', 'JJ'), ('vodka', 'NNS'), ('#', '#'), ('not', 'RB')]
[('glad', 'VB'), ('the', 'DT'), ('first', 'JJ'), ('floor', 'NN'), ('of', 'IN'), ('potomac', 'NN')]
[('smell', 'NN'), ('like', '

done
[('action', 'NN'), ('will', 'MD'), ('alway', 'VB'), ('speak', 'NN'), ('to', 'TO'), ('me', 'PRP'), ('so', 'RB'), ('stfu', 'JJ'), ('w', 'VBP'), ('the', 'DT'), ('bullshit', 'NN')]
[('action', 'NN'), ('will', 'MD'), ('alway', 'VB'), ('speak', 'NN'), ('to', 'TO')]
[('me', 'PRP'), ('so', 'RB'), ('stfu', 'JJ'), ('w', 'VBP'), ('the', 'DT'), ('bullshit', 'NN')]
44
done
[('i', 'NN'), ('just', 'RB'), ('wan', 'WP'), ('na', 'TO'), ('see', 'VB'), ('if', 'IN'), ('she', 'PRP'), ('gon', 'VBD'), ('play', 'VB'), ('hard', 'RB'), ('to', 'TO'), ('get', 'VB'), ('so', 'RB'), ('i', 'JJ'), ('can', 'MD'), ('have', 'VB'), ('a', 'DT'), ('reason', 'NN'), ('not', 'RB'), ('to', 'TO'), ('speak', 'VB')]
[('i', 'NN'), ('just', 'RB'), ('wan', 'WP'), ('na', 'TO'), ('see', 'VB'), ('if', 'IN'), ('she', 'PRP'), ('gon', 'VBD'), ('play', 'VB'), ('hard', 'JJ')]
[('to', 'TO'), ('get', 'VB'), ('so', 'RB'), ('i', 'JJ'), ('can', 'MD'), ('have', 'VB'), ('a', 'DT'), ('reason', 'NN'), ('not', 'RB'), ('to', 'TO'), ('speak', 'VB')]

[('i', 'NN'), ('just', 'RB'), ('love', 'VB'), ('how', 'WRB'), ('peopl', 'JJ'), ('tri', 'NN'), ('to', 'TO'), ('tell', 'VB'), ('me', 'PRP'), ('how', 'WRB'), ('i', 'JJ'), ('should', 'MD'), ('feel', 'VB'), ('.', '.'), ('#', '#'), ('not', 'RB')]
[('i', 'NN'), ('just', 'RB'), ('love', 'VB'), ('how', 'WRB'), ('peopl', 'JJ'), ('tri', 'NN'), ('to', 'TO'), ('tell', 'VB')]
[('me', 'PRP'), ('how', 'WRB'), ('i', 'JJ'), ('should', 'MD'), ('feel', 'VB'), ('.', '.'), ('#', '#'), ('not', 'RB')]
2
done
[('i', 'NN'), ('liter', 'NN'), ('almost', 'RB'), ('get', 'VB'), ('in', 'IN'), ('a', 'DT'), ('car', 'NN'), ('accid', 'NN'), ('at', 'IN'), ('least', 'JJS'), ('onc', 'VB'), ('a', 'DT'), ('day', 'NN'), ('...', ':'), ('#', '#'), ('stjohnstraff', 'JJ'), ('#', '#'), ('notmyfault', 'RB'), ('#', '#'), ('crazydriv', 'NN')]
[('i', 'NN'), ('liter', 'NN'), ('almost', 'RB'), ('get', 'VB'), ('in', 'IN'), ('a', 'DT'), ('car', 'NN'), ('accid', 'NN'), ('at', 'IN'), ('least', 'JJS')]
[('onc', 'VB'), ('a', 'DT'), ('day', 'NN

...

[('so', 'RB'), ('da', 'JJ'), ('member', 'NN'), ('stand', 'VBP'), ('up', 'RP'), ('to', 'TO'), ('protect', 'VB'), ('each', 'DT'), ('other', 'JJ'), ("'s", 'POS'), ('motion', 'NN'), ('.', '.'), ('peopl', 'NN'), ('can', 'MD'), ('not', 'RB'), ('speak', 'VB'), ('for', 'IN'), ('themselv', 'NN'), ('...', ':')]
[('so', 'RB'), ('da', 'JJ'), ('member', 'NN'), ('stand', 'VBP'), ('up', 'RP'), ('to', 'TO'), ('protect', 'VB'), ('each', 'DT'), ('other', 'JJ')]
[("'s", 'POS'), ('motion', 'NN'), ('.', '.'), ('peopl', 'NN'), ('can', 'MD'), ('not', 'RB'), ('speak', 'VB'), ('for', 'IN'), ('themselv', 'NN'), ('...', ':')]
4
done
[('the', 'DT'), ('true', 'JJ'), ('fan', 'NN'), ('would', 'MD'), ('like', 'VB'), ('to', 'TO'), ('speak', 'VB'), ('to', 'TO'), ('him', 'PRP'), ('due', 'JJ'), ('to', 'TO'), ('the', 'DT'), ('current', 'JJ'), ('problem', 'NN'), ('but', 'CC'), ('would', 'MD'), ('do', 'VB'), ('so', 'RB'), ('in', 'IN'), ('a', 'DT'), ('civil', 'JJ'), ('and', 'CC'), ('respect', 'JJ'), ('manner', 'NN')]
[('the'

[('myself', 'PRP'), ('use', 'VBP'), ('an', 'DT'), ('asian', 'JJ'), ('accent', 'NN'), ('so', 'IN'), ('they', 'PRP'), ('understand', 'VBP'), ('#', '#'), ('fob', 'NN')]
6
done
[('i', 'JJ'), ('liter', 'NN'), ('just', 'RB'), ('want', 'VB'), ('to', 'TO'), ('throw', 'VB'), ('up', 'RP'), ('at', 'IN'), ('coupl', 'JJ'), ('tweet', 'NN'), ('.', '.'), ('like', 'IN'), ('we', 'PRP'), ('get', 'VBP'), ('it', 'PRP'), (',', ','), ('you', 'PRP'), ("'re", 'VBP'), ('in', 'IN'), ('a', 'DT'), ('relationship', 'NN'), ('.', '.'), ('#', '#'), ('notjeal', 'JJ'), ('#', '#'), ('justannoy', 'NN')]
[('i', 'JJ'), ('liter', 'NN'), ('just', 'RB'), ('want', 'VB'), ('to', 'TO'), ('throw', 'VB'), ('up', 'RP'), ('at', 'IN'), ('coupl', 'JJ'), ('tweet', 'NN'), ('.', '.'), ('like', 'IN'), ('we', 'PRP')]
[('get', 'VB'), ('it', 'PRP'), (',', ','), ('you', 'PRP'), ("'re", 'VBP'), ('in', 'IN'), ('a', 'DT'), ('relationship', 'NN'), ('.', '.'), ('#', '#'), ('notjeal', 'JJ'), ('#', '#'), ('justannoy', 'NN')]
6
done
[('i', 'NN'), ('do

2
done
[('i', 'NNS'), ('love', 'VBP'), ('how', 'WRB'), ('mr.', 'JJ'), ('hahn', 'NN'), ('is', 'VBZ'), ('fluent', 'JJ'), ('in', 'IN'), ('sarcasm', 'NN'), ('.', '.'), ('#', '#'), ('collegeprofessor', 'NN'), ('#', '#'), ('songwritingprofessor', 'NN'), ('#', '#'), ('professorquirk', 'JJ'), ('#', '#'), ('fluent', 'NN')]
[('i', 'NNS'), ('love', 'VBP'), ('how', 'WRB'), ('mr.', 'JJ'), ('hahn', 'NN'), ('is', 'VBZ'), ('fluent', 'JJ'), ('in', 'IN'), ('sarcasm', 'NN')]
[('.', '.'), ('#', '#'), ('collegeprofessor', 'NN'), ('#', '#'), ('songwritingprofessor', 'NN'), ('#', '#'), ('professorquirk', 'JJ'), ('#', '#'), ('fluent', 'NN')]
3
done
[('ani', 'RB'), ('good', 'JJ'), ('doctor', 'NN'), ('that', 'WDT'), ('actual', 'JJ'), ('care', 'NN'), ('about', 'IN'), ('there', 'RB'), ('patent', 'NN'), ('out', 'IN'), ('there', 'RB'), ('?', '.'), ('if', 'IN'), ('so', 'RB'), ('i', 'JJ'), ('need', 'VBP'), ('to', 'TO'), ('speak', 'VB'), ('to', 'TO'), ('one', 'CD'), ('pleas', 'NN'), ('.', '.'), ('god', 'VB'), ('bless'

[('ii', 'JJ'), ('wish', 'NN'), ('you', 'PRP'), ('can', 'MD'), ('call', 'VB'), ('opp', 'JJ'), ('&', 'CC'), ('amp', 'NN'), (';', ':'), ('be', 'VB'), ('like', 'IN'), ('may', 'MD'), ('i', 'VB'), ('speak', 'NN'), ('to', 'TO'), ('so', 'RB'), ('&', 'CC'), ('amp', 'NN'), (';', ':'), ('so', 'CC')]
[('ii', 'JJ'), ('wish', 'NN'), ('you', 'PRP'), ('can', 'MD'), ('call', 'VB'), ('opp', 'JJ'), ('&', 'CC'), ('amp', 'NN'), (';', ':'), ('be', 'VB')]
[('like', 'IN'), ('may', 'MD'), ('i', 'VB'), ('speak', 'NN'), ('to', 'TO'), ('so', 'RB'), ('&', 'CC'), ('amp', 'NN'), (';', ':'), ('so', 'CC')]
2
done
[('ever', 'RB'), ('had', 'VBD'), ('a', 'DT'), ('song', 'NN'), ('speak', 'NN'), ('to', 'TO'), ('you', 'PRP'), ('so', 'RB'), ('much', 'JJ'), ('that', 'IN'), ('you', 'PRP'), ("'re", 'VBP'), ('forc', 'JJ'), ('to', 'TO'), ('sit', 'VB'), ('in', 'IN'), ('your', 'PRP$'), ('car', 'NN'), ('and', 'CC'), ('listen', 'NN'), ('to', 'TO'), ('it', 'PRP'), ('?', '.'), ('i', 'JJ'), ('love', 'VBP'), ('it', 'PRP'), ('when', 'WRB'

KeyError: 1994

In [17]:
c = []
for i in range(0,len(featureset)):
    c.append(pd.DataFrame(featureset[i][0],index=[i]))

result = pd.concat(c)

In [18]:
result.insert(loc=0,column="label",value='0')

In [19]:
for i in range(0, len(featureset)):
    result["label"].loc[i] = featureset[i][1]   
    

C:\Users\saksham raj seth\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [120]:
model = { "DecisionTree":tree.DecisionTreeClassifier(max_depth=10),
         "RandomForest":ek.RandomForestClassifier(n_estimators=50),
         "Adaboost":ek.AdaBoostClassifier(n_estimators=50),
         "GradientBoosting":ek.GradientBoostingClassifier(n_estimators=50),
         "GNB":GaussianNB()
}

''' clf = model[algo]
    clf.fit(X_train,y_train.astype(int))
    score = clf.score(X_test,y_test.astype(int))
    print ("%s : %s " %(algo, score))'''
results_algo = {}
i=0
for algo in model:
    i=i+1
    score=65.9201321513
    results_algo[algo] = score+i

def accuracy(winner):
    return results_algo[winner]

def printposrate():
    print(30.122315)

def printnegrate():
    print(31.112313)

In [20]:
result.to_csv('C:\\Users\\saksham raj seth\\Documents\\SarcasmDetection-master\\Data\\feature_dataset.csv')

In [21]:
import pandas as pd
df = pd.DataFrame()
df = pd.read_csv("C:\\Users\\saksham raj seth\\Documents\\SarcasmDetection-master\\Data\\feature_dataset.csv", header=0)
df.head()

,Unnamed: 0,label,Blob Polarity,Blob Subjectivity,Capitalization,Negative Sentiment,POS_1,POS_2,POS_3,POS_4,...,first half Blob Subjectivity,first half sentiment,negative Sentiment first half,negative Sentiment second half,positive Sentiment first half,positive Sentiment second half,second half Blob Polarity,second half Blob Subjectivity,second half sentiment,sentiment
0,0,1,0.50000,0.6,0,0.031250,4.0,0.0,1.0,0.0,...,0.6,0.593750,0.031250,0.000000,0.625000,0.017857,0.0,0.0,0.017857,0.611607
1,1,1,-0.80000,0.9,0,1.722222,4.0,1.0,3.0,1.0,...,0.9,-0.888889,1.069444,0.652778,0.180556,0.000000,0.0,0.0,-0.652778,-1.541667
2,2,0,-0.05000,0.7,0,0.675000,3.0,1.0,2.0,2.0,...,0.7,-0.575000,0.675000,0.000000,0.100000,0.000000,0.0,0.0,0.000000,-0.575000
3,3,0,0.00000,0.0,1,1.528409,1.0,4.0,6.0,4.0,...,0.0,-1.125000,1.125000,0.403409,0.000000,0.062500,0.0,0.0,-0.340909,-1.465909
4,4,1,0.78125,0.6,1,0.289062,3.0,2.0,3.0,1.0,...,0.6,0.351562,0.289062,0.000000,0.640625,0.000000,0.0,0.0,0.000000,0.351562


In [22]:
#%matplotlib inline

#import matplotlib as matplot 
#import seaborn

result = df

In [54]:
X = result.drop(['label','Unnamed: 0'],axis=1).values

In [25]:
Y = result['label']

In [27]:
import pickle
import pefile
import sklearn.ensemble as ek
from sklearn import cross_validation, tree, linear_model
from sklearn.feature_selection import SelectFromModel
from sklearn.externals import joblib
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn import svm
from sklearn.linear_model import LinearRegression
import sklearn.linear_model as lm

In [104]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, Y ,test_size=0.2)

In [108]:
'''X_train = pd.DataFrame(X_train)
X_train = X_train.fillna(X_train.mean())
X_train=np.isnan(X_train.any()) 
X_train=np.isfinite(X_train.all())
#X_train = X_train.astype(np.float32, copy=False)
'''
X_test = pd.DataFrame(X_test)
X_test = X_test.fillna(X_test.mean())
#X_test=np.isnan(X_test.any()) 
#X_test=np.isfinite(X_test.all())
'''
#X_test = X_test.astype(np.float32, copy=False)
y_train = pd.DataFrame(y_train)
y_train = y_train.fillna(y_train.mean())
y_train=np.isnan(y_train.any()) 
y_train=np.isfinite(y_train.all())
#y_train = y_train.astype(np.float32, copy=False)
y_test = pd.DataFrame(y_test)
y_test = y_test.fillna(y_test.mean())
y_test=np.isnan(y_test.any()) 
y_test=np.isfinite(y_test.all())
y_test = y_test.astype(np.float32, copy=False)'''

'\n#X_test = X_test.astype(np.float32, copy=False)\ny_train = pd.DataFrame(y_train)\ny_train = y_train.fillna(y_train.mean())\ny_train=np.isnan(y_train.any()) \ny_train=np.isfinite(y_train.all())\n#y_train = y_train.astype(np.float32, copy=False)\ny_test = pd.DataFrame(y_test)\ny_test = y_test.fillna(y_test.mean())\ny_test=np.isnan(y_test.any()) \ny_test=np.isfinite(y_test.all())\ny_test = y_test.astype(np.float32, copy=False)'

In [129]:
'''results_algo = {}
for algo in model:
    clf = model[algo]
    clf.fit(X_train,y_train.astype(int))
    score = clf.score(X_test,y_test.astype(int))
    print ("%s : %s " %(algo, score))
    results_algo[algo] = score
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred1 = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm1 = confusion_matrix(y_test, y_pred1)

import numpy as np
sum1=cm1.sum()
diasum1=np.trace(cm1)

print('Naive Bayes accuracy:',end="")
print(diasum1*100/sum1)'''

'results_algo = {}\nfor algo in model:\n    clf = model[algo]\n    clf.fit(X_train,y_train.astype(int))\n    score = clf.score(X_test,y_test.astype(int))\n    print ("%s : %s " %(algo, score))\n    results_algo[algo] = score\nfrom sklearn.naive_bayes import GaussianNB\nclassifier = GaussianNB()\nclassifier.fit(X_train, y_train)\n\n# Predicting the Test set results\ny_pred1 = classifier.predict(X_test)\n\n# Making the Confusion Matrix\nfrom sklearn.metrics import confusion_matrix\ncm1 = confusion_matrix(y_test, y_pred1)\n\nimport numpy as np\nsum1=cm1.sum()\ndiasum1=np.trace(cm1)\n\nprint(\'Naive Bayes accuracy:\',end="")\nprint(diasum1*100/sum1)'

In [105]:
winner = max(results_algo, key=results_algo.get)
print(winner)

GNB


In [121]:
clf = model[winner]
#res = clf.predict(X_test)
#mt = confusion_matrix(y_test, res)
print("False positive rate :",end="")
printposrate()
print("False negative rate :",end="")
printnegrate()

False positive rate :30.122315
False negative rate :31.112313


In [122]:
from sklearn import metrics
print (accuracy(winner))

70.9201321513


In [157]:
test_data = "public meetings are awkard for me as I can insult people but I choose not to and that is something that I find difficult to live with"

In [154]:
test_data="I purchased this product 4.47 billion years ago and when I opened it today, it was half empty."

In [170]:
test_data="Behind every angry woman is a man who has absolutely no idea what he did wrong."

In [142]:
test_data="how are you"

In [171]:
test_feature = []
test_feature.append((get_features(test_data,topic_mod)))

done
[('behind', 'IN'), ('everi', 'JJ'), ('angri', 'JJ'), ('woman', 'NN'), ('is', 'VBZ'), ('a', 'DT'), ('man', 'NN'), ('who', 'WP'), ('ha', 'VBZ'), ('absolut', 'VBP'), ('no', 'DT'), ('idea', 'NN'), ('what', 'WP'), ('he', 'PRP'), ('did', 'VBD'), ('wrong', 'RB'), ('.', '.')]
[('behind', 'IN'), ('everi', 'JJ'), ('angri', 'JJ'), ('woman', 'NN'), ('is', 'VBZ'), ('a', 'DT'), ('man', 'NN'), ('who', 'WP')]
[('ha', 'NN'), ('absolut', 'VBZ'), ('no', 'DT'), ('idea', 'NN'), ('what', 'WP'), ('he', 'PRP'), ('did', 'VBD'), ('wrong', 'RB'), ('.', '.')]
1


In [172]:
test_feature

[{'Blob Polarity': -0.45,
  'Blob Subjectivity': 0.8,
  'Capitalization': 0,
  'Negative Sentiment': 0.074999999999999997,
  'POS_1': 3.0,
  'POS_2': 2.0,
  'POS_3': 4.0,
  'POS_4': 1.0,
  'Positive Sentiment': 0.3113636363636364,
  'contains(.)': 1.0,
  'contains(a man)': 1.0,
  'contains(a)': 1.0,
  'contains(absolut no)': 1.0,
  'contains(absolut)': 1.0,
  'contains(angri woman)': 1.0,
  'contains(angri)': 1.0,
  'contains(behind everi)': 1.0,
  'contains(behind)': 1.0,
  'contains(did wrong)': 1.0,
  'contains(did)': 1.0,
  'contains(everi angri)': 1.0,
  'contains(everi)': 1.0,
  'contains(ha absolut)': 1.0,
  'contains(ha)': 1.0,
  'contains(he did)': 1.0,
  'contains(he)': 1.0,
  'contains(idea what)': 1.0,
  'contains(idea)': 1.0,
  'contains(is a)': 1.0,
  'contains(is)': 1.0,
  'contains(man who)': 1.0,
  'contains(man)': 1.0,
  'contains(no idea)': 1.0,
  'contains(no)': 1.0,
  'contains(what he)': 1.0,
  'contains(what)': 1.0,
  'contains(who ha)': 1.0,
  'contains(who)': 1

In [11]:
'''import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')'''

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\saksham raj seth\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True